In [1]:
import numpy as np
import h5py 
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [2]:
N = input('Ingrese el número de archivo deseado para el snapshot 125 de la multidark galaxy:')

Ingrese el número de archivo deseado para el snapshot 125 de la multidark galaxy:001


In [3]:
Archivo = f"gal_125_SAG-7.128_BOX_{N}.hdf5"

In [4]:
with h5py.File(Archivo, "r") as archivo:
    for datos in archivo.keys():
        print(datos)

FeH_hot_halo
FeH_stars
ForestID
GalaxyHostID
GalaxyStaticID
Galaxy_Type
Halo
HaloID
InstantSfrBulge
InstantSfrQuies
MZ_ICstars
MZ_gas_bulge
MZ_gas_disk
MZ_hot_halo
MZ_stars_bulge
MZ_stars_disk
M_ICstars
M_gas_bulge
M_gas_disk
M_hot
M_star_bulge
M_star_disk
Magnitudes
MainHaloID
Mbh
OH_gas_disk_bulge
Rhalf_bulge
Rhalf_disk
SFR
SFR_bulge
SFR_bulge_DI
T_stars
Vx
Vy
Vz
X
Y
Z


In [5]:
with h5py.File(Archivo, "r") as archivo:
    MainID = archivo['MainHaloID'][:].ravel()
    HaloID = archivo['HaloID'][:].ravel()
    R200 = archivo['Halo/R200c'][:].ravel()*0.674
    G_type = archivo['Galaxy_Type'][:].ravel()
    r = archivo['Magnitudes/Mag_rS'][:].ravel()
    x = archivo['X'][:].ravel()*0.674
    y = archivo['Y'][:].ravel()*0.674
    z = archivo['Z'][:].ravel()*0.674
    Vx = archivo['Vx'][:].ravel()
    Vy = archivo['Vy'][:].ravel()
    Vz = archivo['Vz'][:].ravel()   
    Vmax = archivo['Halo/Vmax'][:].ravel()

In [31]:
# Crear un DataFrame de pandas con el conjunto de datos
data = pd.DataFrame({'MainHaloID':MainID, 'HaloID': HaloID, 'Galaxy_Type': G_type, 'r':r, 'X':x, 'Y':y, 'Z':z, 'Vx':Vx, 'Vy':Vy, 'Vz':Vz, 'R200c':R200})
data.to_csv('Simulation.csv')
# Filtrar por número de galaxias pertenecientes a cada halo
#groups_filter = data.groupby('MainHaloID').filter(lambda x: 3 < len(x) <= 10 and x['r'].max() <= -18 and (x['r'].max() - x['r'].min()) <= 3)
#groups_filter

In [9]:
# Filtrar por número de galaxias pertenecientes a cada halo
groups_filter = data.groupby('MainHaloID').filter(lambda x: 3 < len(x) <= 10 and x['r'].max() <= -18 and (x['r'].max() - x['r'].min()) <= 3)
groups_filter

ValueError: operands could not be broadcast together with shapes (150803,3) (7,3) 

In [16]:
#Identificar la galaxia host y las galaxias satélite en cada grupo
def calcular_dispersion(group):
    galaxia_host = group[group['Galaxy_Type'] == 0]
    galaxias_satelite = group[group['Galaxy_Type'].isin([1, 2])]

    #Restar las velocidades de las galaxias satélite respecto a la galaxia host
    dispersion_velocidades = galaxias_satelite[['Vx', 'Vy', 'Vz']] - galaxia_host[['Vx', 'Vy', 'Vz']].values[0]

    #Calcular la magnitud de la dispersión de velocidades
    dispersion_magnitud = dispersion_velocidades.apply(lambda x: (x**2).sum()**0.5, axis=1)

    return dispersion_magnitud.mean()

#Aplica la función a cada grupo y obtén la dispersión de velocidades promedio
dispersion_promedio = groups_filter.groupby('MainHaloID').apply(calcular_dispersion)

print(dispersion_promedio)


MainHaloID
12576613642    117.431639
12577118276    157.378902
12577195475    120.767280
12588457573    169.390297
12591398490    115.554758
12608836633    191.767053
12617340271    158.716254
12618761495    374.799718
12620442527    177.829216
12621913489    177.268810
12624327172    143.945472
12625608302    156.759270
12634294437    198.168592
12637222261     86.677120
12647919999    178.284237
12662091357    192.696376
12662402481    182.851811
12670348906    168.492034
12683359802    181.750205
12683365296    302.041882
12689133732    113.904080
dtype: float64


In [17]:
groups_filter['Dispersion'] = groups_filter['MainHaloID'].map(dispersion_promedio)
groups_filter

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Dispersion
83846,12576613642,12576613642,0,-18.001469,16442.925781,544109.250000,461259.343750,-70.010002,-265.839996,73.839996,69.868484,117.431639
83847,12576613642,12576613642,2,-18.065386,16443.025391,544132.500000,461274.406250,-102.055527,-282.519073,95.552750,41.217846,117.431639
83848,12576613642,12576613643,1,-18.553612,16474.947266,544076.000000,461275.812500,-243.520004,-257.549988,72.970001,51.528862,117.431639
83849,12576613642,12576613644,1,-18.456289,16530.968750,544135.062500,461281.281250,-111.690002,-394.029999,52.750000,50.253357,117.431639
89511,12577118276,12577118276,0,-20.831751,36086.917969,627973.625000,314403.031250,-3.290000,2.840000,-482.459991,102.495743,157.378902
...,...,...,...,...,...,...,...,...,...,...,...,...
1258238,12683365296,12683365298,1,-18.162418,548610.562500,332357.375000,3590.189453,-450.019989,326.290009,-172.240005,50.513847,302.041882
1324630,12689133732,12689133732,0,-18.864124,586386.437500,85583.703125,139164.125000,-117.089996,-218.880005,92.010002,82.677338,113.904080
1324631,12689133732,12689133732,2,-19.025618,586370.187500,85577.523438,139156.390625,-111.420799,-95.021873,124.273605,51.819508,113.904080
1324632,12689133732,12689133732,2,-18.113550,586404.437500,85568.617188,139174.281250,-121.382492,-308.086975,111.935471,44.560883,113.904080


In [18]:
cg = groups_filter.groupby('MainHaloID').filter(lambda x: x['Dispersion'].mean() < 1000)
cg

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Dispersion
83846,12576613642,12576613642,0,-18.001469,16442.925781,544109.250000,461259.343750,-70.010002,-265.839996,73.839996,69.868484,117.431639
83847,12576613642,12576613642,2,-18.065386,16443.025391,544132.500000,461274.406250,-102.055527,-282.519073,95.552750,41.217846,117.431639
83848,12576613642,12576613643,1,-18.553612,16474.947266,544076.000000,461275.812500,-243.520004,-257.549988,72.970001,51.528862,117.431639
83849,12576613642,12576613644,1,-18.456289,16530.968750,544135.062500,461281.281250,-111.690002,-394.029999,52.750000,50.253357,117.431639
89511,12577118276,12577118276,0,-20.831751,36086.917969,627973.625000,314403.031250,-3.290000,2.840000,-482.459991,102.495743,157.378902
...,...,...,...,...,...,...,...,...,...,...,...,...
1258238,12683365296,12683365298,1,-18.162418,548610.562500,332357.375000,3590.189453,-450.019989,326.290009,-172.240005,50.513847,302.041882
1324630,12689133732,12689133732,0,-18.864124,586386.437500,85583.703125,139164.125000,-117.089996,-218.880005,92.010002,82.677338,113.904080
1324631,12689133732,12689133732,2,-19.025618,586370.187500,85577.523438,139156.390625,-111.420799,-95.021873,124.273605,51.819508,113.904080
1324632,12689133732,12689133732,2,-18.113550,586404.437500,85568.617188,139174.281250,-121.382492,-308.086975,111.935471,44.560883,113.904080


In [ ]:
def scatter_plot(ax, eje1, eje2, title):
    ax.scatter(cg.head(4)[eje1], cg.head(4)[eje2], marker= '*')
    ax.set_title(title)

#Crear una figura y ejes para los subgráficos
#fig, axs = plt.subplots(1, 3, figsize=(16, 4))

#Crear el primer gráfico con el conjunto de datos 
#scatter_plot(axs[0], 'X', 'Y', 'Plano XY')

#Crear el segundo gráfico con el conjunto de datos 
#scatter_plot(axs[1], 'Z', 'Y', 'Plano YZ')

#Crear el tercer gráfico con el conjunto de datos 
#scatter_plot(axs[2], 'X', 'Z', 'Plano XZ')



In [ ]:
# Crear una figura con tres subgráficos en una fila
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Iterar sobre las filas del DataFrame
for index, row in cg.iterrows():
    # Primer gráfico: Posición X vs. Posición Y
    axs[0].scatter(cg['X'], cg['Y'])
    axs[0].set_title('Posición X vs. Posición Y')
    axs[0].set_xlabel('Posición X')
    axs[0].set_ylabel('Posición Y')
    # Agregar el círculo centrado en la galaxia host con el radio especificado
    if row['Galaxy_Type'] == 0:
        circle = plt.Circle((row['X'], row['Y']), 2*row['R200c'], color='r', fill=False)
        axs[0].add_patch(circle)

    # Segundo gráfico: Posición Y vs. Posición Z
    axs[1].scatter(cg['Y'], cg['Z'])
    axs[1].set_title('Posición Y vs. Posición Z')
    axs[1].set_xlabel('Posición Y')
    axs[1].set_ylabel('Posición Z')
    # Agregar el círculo centrado en la galaxia host con el radio especificado
    if row['Galaxy_Type'] == 0:
        circle = plt.Circle((row['Y'], row['Z']), 2*row['R200c'], color='r', fill=False)
        axs[1].add_patch(circle)

    # Tercer gráfico: Posición X vs. Posición Z
    axs[2].scatter(cg['X'], cg['Z'])
    axs[2].set_title('Posición X vs. Posición Z')
    axs[2].set_xlabel('Posición X')
    axs[2].set_ylabel('Posición Z')
    # Agregar el círculo centrado en la galaxia host con el radio especificado
    if row['Galaxy_Type'] == 0:
        circle = plt.Circle((row['X'], row['Z']), 2*row['R200c'], color='r', fill=False)
        axs[2].add_patch(circle)

# Ajustar el diseño de los subgráficos
plt.tight_layout()

# Mostrar los gráficos
plt.show()

In [19]:
cg['Distancia XY'] = np.sqrt(cg['X']**2 + cg['Y']**2)
cg['Distancia YZ'] = np.sqrt(cg['Y']**2 + cg['Z']**2)
cg['Distancia XZ'] = np.sqrt(cg['X']**2 + cg['Z']**2)
cg

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Dispersion,Distancia XY,Distancia YZ,Distancia XZ
83846,12576613642,12576613642,0,-18.001469,16442.925781,544109.250000,461259.343750,-70.010002,-265.839996,73.839996,69.868484,117.431639,544357.6250,713312.750000,461552.31250
83847,12576613642,12576613642,2,-18.065386,16443.025391,544132.500000,461274.406250,-102.055527,-282.519073,95.552750,41.217846,117.431639,544380.8750,713340.250000,461567.37500
83848,12576613642,12576613643,1,-18.553612,16474.947266,544076.000000,461275.812500,-243.520004,-257.549988,72.970001,51.528862,117.431639,544325.3750,713298.000000,461569.90625
83849,12576613642,12576613644,1,-18.456289,16530.968750,544135.062500,461281.281250,-111.690002,-394.029999,52.750000,50.253357,117.431639,544386.1250,713346.625000,461577.37500
89511,12577118276,12577118276,0,-20.831751,36086.917969,627973.625000,314403.031250,-3.290000,2.840000,-482.459991,102.495743,157.378902,629009.6875,702282.125000,316467.28125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258238,12683365296,12683365298,1,-18.162418,548610.562500,332357.375000,3590.189453,-450.019989,326.290009,-172.240005,50.513847,302.041882,641432.0000,332376.750000,548622.31250
1324630,12689133732,12689133732,0,-18.864124,586386.437500,85583.703125,139164.125000,-117.089996,-218.880005,92.010002,82.677338,113.904080,592599.0000,163374.484375,602673.75000
1324631,12689133732,12689133732,2,-19.025618,586370.187500,85577.523438,139156.390625,-111.420799,-95.021873,124.273605,51.819508,113.904080,592582.0625,163364.671875,602656.18750
1324632,12689133732,12689133732,2,-18.113550,586404.437500,85568.617188,139174.281250,-121.382492,-308.086975,111.935471,44.560883,113.904080,592614.6875,163375.234375,602693.68750


In [20]:
cg['m_rx'] = cg['r'] + 5*np.log10(cg['Distancia XY']*1000) - 5
cg['m_ry'] = cg['r'] + 5*np.log10(cg['Distancia YZ']*1000) - 5
cg['m_rz'] = cg['r'] + 5*np.log10(cg['Distancia XZ']*1000) - 5
cg

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Dispersion,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz
83846,12576613642,12576613642,0,-18.001469,16442.925781,544109.250000,461259.343750,-70.010002,-265.839996,73.839996,69.868484,117.431639,544357.6250,713312.750000,461552.31250,20.677956,21.264931,20.319637
83847,12576613642,12576613642,2,-18.065386,16443.025391,544132.500000,461274.406250,-102.055527,-282.519073,95.552750,41.217846,117.431639,544380.8750,713340.250000,461567.37500,20.614126,21.201094,20.255789
83848,12576613642,12576613643,1,-18.553612,16474.947266,544076.000000,461275.812500,-243.520004,-257.549988,72.970001,51.528862,117.431639,544325.3750,713298.000000,461569.90625,20.125683,20.712742,19.767578
83849,12576613642,12576613644,1,-18.456289,16530.968750,544135.062500,461281.281250,-111.690002,-394.029999,52.750000,50.253357,117.431639,544386.1250,713346.625000,461577.37500,20.223246,20.810217,19.864935
89511,12577118276,12577118276,0,-20.831751,36086.917969,627973.625000,314403.031250,-3.290000,2.840000,-482.459991,102.495743,157.378902,629009.6875,702282.125000,316467.28125,18.161535,18.400808,16.669893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258238,12683365296,12683365298,1,-18.162418,548610.562500,332357.375000,3590.189453,-450.019989,326.290009,-172.240005,50.513847,302.041882,641432.0000,332376.750000,548622.31250,20.873337,19.445732,20.533947
1324630,12689133732,12689133732,0,-18.864124,586386.437500,85583.703125,139164.125000,-117.089996,-218.880005,92.010002,82.677338,113.904080,592599.0000,163374.484375,602673.75000,19.999683,17.201797,20.036289
1324631,12689133732,12689133732,2,-19.025618,586370.187500,85577.523438,139156.390625,-111.420799,-95.021873,124.273605,51.819508,113.904080,592582.0625,163364.671875,602656.18750,19.838129,17.040174,19.874727
1324632,12689133732,12689133732,2,-18.113550,586404.437500,85568.617188,139174.281250,-121.382492,-308.086975,111.935471,44.560883,113.904080,592614.6875,163375.234375,602693.68750,20.750315,17.952383,20.786936


In [21]:
#Para seleccionar solo el R200 de la galaxia host
cg['Host_R200'] = cg.groupby('MainHaloID')['R200c'].transform('first')
cg['Host_Distancia XY'] = cg.groupby('MainHaloID')['Distancia XY'].transform('first')
cg['Host_Distancia YZ'] = cg.groupby('MainHaloID')['Distancia YZ'].transform('first')
cg['Host_Distancia XZ'] = cg.groupby('MainHaloID')['Distancia XZ'].transform('first')

# Calcula el tamaño angular para cada dirección usando el R200 de la galaxia host en su grupo
cg['Tamaño Angular del grupo XY (arcsec)'] = 2*np.arctan((2*cg['Host_R200'])/(cg['Host_Distancia XY']))*3600
cg['Tamaño Angular del grupo YZ (arcsec)'] = 2*np.arctan((2*cg['Host_R200'])/(cg['Host_Distancia YZ']))*3600
cg['Tamaño Angular del grupo XZ (arcsec)'] = 2*np.arctan((2*cg['Host_R200'])/(cg['Host_Distancia XZ']))*3600

cg.drop('Host_R200', axis=1, inplace=True)
cg.drop('Host_Distancia XY', axis=1, inplace=True)
cg.drop('Host_Distancia YZ', axis=1, inplace=True)
cg.drop('Host_Distancia XZ', axis=1, inplace=True)
cg

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,...,Dispersion,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec)
83846,12576613642,12576613642,0,-18.001469,16442.925781,544109.250000,461259.343750,-70.010002,-265.839996,73.839996,...,117.431639,544357.6250,713312.750000,461552.31250,20.677956,21.264931,20.319637,1.848245,1.410470,2.179831
83847,12576613642,12576613642,2,-18.065386,16443.025391,544132.500000,461274.406250,-102.055527,-282.519073,95.552750,...,117.431639,544380.8750,713340.250000,461567.37500,20.614126,21.201094,20.255789,1.848245,1.410470,2.179831
83848,12576613642,12576613643,1,-18.553612,16474.947266,544076.000000,461275.812500,-243.520004,-257.549988,72.970001,...,117.431639,544325.3750,713298.000000,461569.90625,20.125683,20.712742,19.767578,1.848245,1.410470,2.179831
83849,12576613642,12576613644,1,-18.456289,16530.968750,544135.062500,461281.281250,-111.690002,-394.029999,52.750000,...,117.431639,544386.1250,713346.625000,461577.37500,20.223246,20.810217,19.864935,1.848245,1.410470,2.179831
89511,12577118276,12577118276,0,-20.831751,36086.917969,627973.625000,314403.031250,-3.290000,2.840000,-482.459991,...,157.378902,629009.6875,702282.125000,316467.28125,18.161535,18.400808,16.669893,2.346448,2.101632,4.663795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258238,12683365296,12683365298,1,-18.162418,548610.562500,332357.375000,3590.189453,-450.019989,326.290009,-172.240005,...,302.041882,641432.0000,332376.750000,548622.31250,20.873337,19.445732,20.533947,2.584185,4.986802,3.021405
1324630,12689133732,12689133732,0,-18.864124,586386.437500,85583.703125,139164.125000,-117.089996,-218.880005,92.010002,...,113.904080,592599.0000,163374.484375,602673.75000,19.999683,17.201797,20.036289,2.009038,7.287265,1.975453
1324631,12689133732,12689133732,2,-19.025618,586370.187500,85577.523438,139156.390625,-111.420799,-95.021873,124.273605,...,113.904080,592582.0625,163364.671875,602656.18750,19.838129,17.040174,19.874727,2.009038,7.287265,1.975453
1324632,12689133732,12689133732,2,-18.113550,586404.437500,85568.617188,139174.281250,-121.382492,-308.086975,111.935471,...,113.904080,592614.6875,163375.234375,602693.68750,20.750315,17.952383,20.786936,2.009038,7.287265,1.975453


In [22]:
def brillo(group):
    brillo_xy = -2.5*np.log10((np.sum(10**(-0.4*group['m_rx'])))/(np.pi*group['Tamaño Angular del grupo XY (arcsec)']**2))
    brillo_yz = -2.5*np.log10((np.sum(10**(-0.4*group['m_ry'])))/(np.pi*group['Tamaño Angular del grupo YZ (arcsec)']**2))
    brillo_xz = -2.5*np.log10((np.sum(10**(-0.4*group['m_rz'])))/(np.pi*group['Tamaño Angular del grupo XZ (arcsec)']**2))
    #print(group["m_rx"],group["Tamaño Angular del grupo XY (arcsec)"])
    brillo_promedio = (brillo_xy+brillo_yz+brillo_xz)/3

    return(brillo_promedio)

brillo_s = cg.groupby('MainHaloID').apply(brillo)
brillo_s

cg['Brillo Superficial'] = cg['MainHaloID'].map(brillo_s)
cg


,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,...,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec),Brillo Superficial
83846,12576613642,12576613642,0,-18.001469,16442.925781,544109.250000,461259.343750,-70.010002,-265.839996,73.839996,...,544357.6250,713312.750000,461552.31250,20.677956,21.264931,20.319637,1.848245,1.410470,2.179831,NaN
83847,12576613642,12576613642,2,-18.065386,16443.025391,544132.500000,461274.406250,-102.055527,-282.519073,95.552750,...,544380.8750,713340.250000,461567.37500,20.614126,21.201094,20.255789,1.848245,1.410470,2.179831,NaN
83848,12576613642,12576613643,1,-18.553612,16474.947266,544076.000000,461275.812500,-243.520004,-257.549988,72.970001,...,544325.3750,713298.000000,461569.90625,20.125683,20.712742,19.767578,1.848245,1.410470,2.179831,NaN
83849,12576613642,12576613644,1,-18.456289,16530.968750,544135.062500,461281.281250,-111.690002,-394.029999,52.750000,...,544386.1250,713346.625000,461577.37500,20.223246,20.810217,19.864935,1.848245,1.410470,2.179831,NaN
89511,12577118276,12577118276,0,-20.831751,36086.917969,627973.625000,314403.031250,-3.290000,2.840000,-482.459991,...,629009.6875,702282.125000,316467.28125,18.161535,18.400808,16.669893,2.346448,2.101632,4.663795,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258238,12683365296,12683365298,1,-18.162418,548610.562500,332357.375000,3590.189453,-450.019989,326.290009,-172.240005,...,641432.0000,332376.750000,548622.31250,20.873337,19.445732,20.533947,2.584185,4.986802,3.021405,NaN
1324630,12689133732,12689133732,0,-18.864124,586386.437500,85583.703125,139164.125000,-117.089996,-218.880005,92.010002,...,592599.0000,163374.484375,602673.75000,19.999683,17.201797,20.036289,2.009038,7.287265,1.975453,NaN
1324631,12689133732,12689133732,2,-19.025618,586370.187500,85577.523438,139156.390625,-111.420799,-95.021873,124.273605,...,592582.0625,163364.671875,602656.18750,19.838129,17.040174,19.874727,2.009038,7.287265,1.975453,NaN
1324632,12689133732,12689133732,2,-18.113550,586404.437500,85568.617188,139174.281250,-121.382492,-308.086975,111.935471,...,592614.6875,163375.234375,602693.68750,20.750315,17.952383,20.786936,2.009038,7.287265,1.975453,NaN


In [23]:
def brillo(group):
    brillo_xy = -2.5 * np.log10((np.sum(10 ** (-0.4 * group['m_rx']))) / (np.pi * group['Tamaño Angular del grupo XY (arcsec)'] ** 2))
    brillo_yz = -2.5 * np.log10((np.sum(10 ** (-0.4 * group['m_ry']))) / (np.pi * group['Tamaño Angular del grupo YZ (arcsec)'] ** 2))
    brillo_xz = -2.5 * np.log10((np.sum(10 ** (-0.4 * group['m_rz']))) / (np.pi * group['Tamaño Angular del grupo XZ (arcsec)'] ** 2))

    # Calcular el brillo promedio y asignar el resultado a cada fila del grupo
    brillo_promedio = (brillo_xy + brillo_yz + brillo_xz) / 3
    group['Brillo Superficial'] = brillo_promedio

    return group

# Aplicar la función a cada grupo y luego combinar los resultados
cg = cg.groupby('MainHaloID').apply(brillo)
cg

/tmp/ipykernel_6733/3415725933.py:13: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  cg = cg.groupby('MainHaloID').apply(brillo)


,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,...,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec),Brillo Superficial
83846,12576613642,12576613642,0,-18.001469,16442.925781,544109.250000,461259.343750,-70.010002,-265.839996,73.839996,...,544357.6250,713312.750000,461552.31250,20.677956,21.264931,20.319637,1.848245,1.410470,2.179831,21.455551
83847,12576613642,12576613642,2,-18.065386,16443.025391,544132.500000,461274.406250,-102.055527,-282.519073,95.552750,...,544380.8750,713340.250000,461567.37500,20.614126,21.201094,20.255789,1.848245,1.410470,2.179831,21.455551
83848,12576613642,12576613643,1,-18.553612,16474.947266,544076.000000,461275.812500,-243.520004,-257.549988,72.970001,...,544325.3750,713298.000000,461569.90625,20.125683,20.712742,19.767578,1.848245,1.410470,2.179831,21.455551
83849,12576613642,12576613644,1,-18.456289,16530.968750,544135.062500,461281.281250,-111.690002,-394.029999,52.750000,...,544386.1250,713346.625000,461577.37500,20.223246,20.810217,19.864935,1.848245,1.410470,2.179831,21.455551
89511,12577118276,12577118276,0,-20.831751,36086.917969,627973.625000,314403.031250,-3.290000,2.840000,-482.459991,...,629009.6875,702282.125000,316467.28125,18.161535,18.400808,16.669893,2.346448,2.101632,4.663795,20.870413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258238,12683365296,12683365298,1,-18.162418,548610.562500,332357.375000,3590.189453,-450.019989,326.290009,-172.240005,...,641432.0000,332376.750000,548622.31250,20.873337,19.445732,20.533947,2.584185,4.986802,3.021405,20.955683
1324630,12689133732,12689133732,0,-18.864124,586386.437500,85583.703125,139164.125000,-117.089996,-218.880005,92.010002,...,592599.0000,163374.484375,602673.75000,19.999683,17.201797,20.036289,2.009038,7.287265,1.975453,21.428192
1324631,12689133732,12689133732,2,-19.025618,586370.187500,85577.523438,139156.390625,-111.420799,-95.021873,124.273605,...,592582.0625,163364.671875,602656.18750,19.838129,17.040174,19.874727,2.009038,7.287265,1.975453,21.428192
1324632,12689133732,12689133732,2,-18.113550,586404.437500,85568.617188,139174.281250,-121.382492,-308.086975,111.935471,...,592614.6875,163375.234375,602693.68750,20.750315,17.952383,20.786936,2.009038,7.287265,1.975453,21.428192


In [24]:
cg = cg.groupby('MainHaloID')(lambda x: x['Brillo Superficial'] < 26.33)
cg

TypeError: filter function returned a Series, but expected a scalar bool

In [28]:
cg.loc[cg['Brillo Superficial'] < 26.33]

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,...,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec),Brillo Superficial
83846,12576613642,12576613642,0,-18.001469,16442.925781,544109.250000,461259.343750,-70.010002,-265.839996,73.839996,...,544357.6250,713312.750000,461552.31250,20.677956,21.264931,20.319637,1.848245,1.410470,2.179831,21.455551
83847,12576613642,12576613642,2,-18.065386,16443.025391,544132.500000,461274.406250,-102.055527,-282.519073,95.552750,...,544380.8750,713340.250000,461567.37500,20.614126,21.201094,20.255789,1.848245,1.410470,2.179831,21.455551
83848,12576613642,12576613643,1,-18.553612,16474.947266,544076.000000,461275.812500,-243.520004,-257.549988,72.970001,...,544325.3750,713298.000000,461569.90625,20.125683,20.712742,19.767578,1.848245,1.410470,2.179831,21.455551
83849,12576613642,12576613644,1,-18.456289,16530.968750,544135.062500,461281.281250,-111.690002,-394.029999,52.750000,...,544386.1250,713346.625000,461577.37500,20.223246,20.810217,19.864935,1.848245,1.410470,2.179831,21.455551
89511,12577118276,12577118276,0,-20.831751,36086.917969,627973.625000,314403.031250,-3.290000,2.840000,-482.459991,...,629009.6875,702282.125000,316467.28125,18.161535,18.400808,16.669893,2.346448,2.101632,4.663795,20.870413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258238,12683365296,12683365298,1,-18.162418,548610.562500,332357.375000,3590.189453,-450.019989,326.290009,-172.240005,...,641432.0000,332376.750000,548622.31250,20.873337,19.445732,20.533947,2.584185,4.986802,3.021405,20.955683
1324630,12689133732,12689133732,0,-18.864124,586386.437500,85583.703125,139164.125000,-117.089996,-218.880005,92.010002,...,592599.0000,163374.484375,602673.75000,19.999683,17.201797,20.036289,2.009038,7.287265,1.975453,21.428192
1324631,12689133732,12689133732,2,-19.025618,586370.187500,85577.523438,139156.390625,-111.420799,-95.021873,124.273605,...,592582.0625,163364.671875,602656.18750,19.838129,17.040174,19.874727,2.009038,7.287265,1.975453,21.428192
1324632,12689133732,12689133732,2,-18.113550,586404.437500,85568.617188,139174.281250,-121.382492,-308.086975,111.935471,...,592614.6875,163375.234375,602693.68750,20.750315,17.952383,20.786936,2.009038,7.287265,1.975453,21.428192


In [30]:
candidatos_fase3 = cg.loc[cg['Galaxy_Type'] == 0]
candidatos_fase3.to_csv('candidatos_fase3.csv')